# Comparison Setup, loads TSPs and creates pandas dataframe

## Imports

In [14]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import tsplib95
import numpy as np
from multiprocess import Pool
import signal
import tqdm

from wsp import tsp, ds, util # TODO: refine imports

TREE_TYPE = ds.PKPRQuadTree
BF_THRESHOLD = 19
S_FACTOR = 2.0 # NOTE: if this value is changed then the kernel ought to be restarted

## Loading Euclidean TSPs

In [15]:
all_problems : list[tsplib95.models.StandardProblem] = []

for file in sorted(os.listdir("ALL_tsp")): # Loop through every tsp
    if not file.endswith(".tsp"):
        continue
    problem = tsplib95.load(f"ALL_tsp/{file}")
    if problem.edge_weight_type != "EUC_2D": # Skip non-Euclidean TSPs
        continue # TODO: include ATT and GEO and maybe EUC_3D
    
    all_problems.append(problem)
    print(f"Added {problem.name}")

print("Found", len(all_problems), "euclidean TSPs")

Added a280
Added ara238025
Added bby34656
Added bbz25234
Added bch2762
Added bck2217
Added bcl380
Added beg3293
Added berlin52
Added bgb4355
Added bgd4396
Added bgf4475
Added bier127
Added bm33708
Added bna56769
Added bnd7168
Added boa28924
Added brd14051
Added bva2144
Added ca4663
Added ch130
Added ch150
Added ch71009
Added courbet180K
Added d1291
Added d15112
Added d1655
Added d18512
Added d198
Added d2103
Added d493
Added d657
Added dan59296
Added dbj2924
Added dca1389
Added dcb2086
Added dcc1911
Added dea2382
Added dga9698
Added dhb3386
Added dj38
Added dja1436
Added djb2036
Added djc1785
Added dka1376
Added dkc3938
Added dkd1973
Added dke3097
Added dkf3954
Added dkg813
Added dlb3694
Added earring200K
Added eg7146
Added ei8246
Added eil101
Added eil51
Added eil76
Added fdp3256
Added fea5557
Added fht47608
Added fi10639
Added fjr3672
Added fjs3649
Added fl1400
Added fl1577
Added fl3795
Added fl417
Added fma21553
Added fna52057
Added fnb1615
Added fnc19402
Added fnl4461
Added fqm5087

## Setting up data science & Save Default Values into dataframe

In [18]:
# Create a data frame with columns for the name opt solution, nearest neighbor solution, and untouched solution

if os.path.exists("results.pkl"):
    df = pd.read_pickle("results.pkl")
else:
    df = pd.DataFrame(columns=['name', 'num_points', f'wsp_count_{S_FACTOR}', 'opt_len', 'nn_len', 'ls_len', 'ls_q_len', 'untouched_len', 'nwsp5_len'])
    df.name = pd.Series(sorted([problem.name for problem in all_problems]))
    df.set_index("name", inplace = True)
df.loc["brd14051", "opt_len"] = None
df.loc["d18512", "opt_len"] = None
df.loc["pla33810", "opt_len"] = None
print("null percents", df.isnull().sum() * 100 / len(df), sep="\n")
df

null percents
num_points        0.478469
opt_len          28.708134
nn_len            1.435407
untouched_len     0.478469
nwsp5_2_len      68.899522
nwsp5_len        91.866029
wsp_count_2.0     0.478469
ls_len           64.114833
ls_q_len         64.114833
dtype: float64


,num_points,opt_len,nn_len,untouched_len,nwsp5_2_len,nwsp5_len,wsp_count_2.0,ls_len,ls_q_len
name,,,,,,,,,
a280,280,2.586770e+03,3.161004e+03,4.847061e+03,NaN,NaN,2571,2769.255661,2956.917525
ara238025,238025,NaN,7.591498e+05,1.176390e+06,NaN,NaN,3032365,NaN,NaN
bby34656,34656,NaN,1.253833e+05,1.966541e+05,NaN,NaN,412676,NaN,NaN
bbz25234,25234,NaN,8.772034e+04,1.414553e+05,NaN,NaN,299703,NaN,NaN
bch2762,2762,NaN,1.073362e+04,1.624823e+04,NaN,NaN,28611,NaN,NaN
...,...,...,...,...,...,...,...,...,...
courbet180K,180000,7.894985e+06,9.407409e+06,1.868348e+09,NaN,NaN,3440440,NaN,NaN
earring200K,200000,8.178053e+06,9.762381e+06,2.188609e+09,NaN,NaN,3831148,NaN,NaN
lrb744710,744710,NaN,NaN,1.700243e+08,NaN,NaN,10051610,NaN,NaN


## Running Necessary Math On Each Problem

In [19]:
problems = list(filter(lambda problem: problem.dimension < 50000, all_problems)) # Filter out problems with too many points

# fig, ax = plt.subplots(1, 2, figsize=(12,6))
ax = np.array([None, None])
i = 0
def setup(problem : tsplib95.models.StandardProblem):
    # print(problem.name) # i
    # ax[0].clear()
    # ax[1].clear()
    updates = {}

    points = [ds.Point(*problem.node_coords[i]) for i in problem.get_nodes()]
    
    # if len(points) > 150000: return None # saves time by skipping before creating the tree

    ts_problem : tsp.TravellingSalesmanProblem[TREE_TYPE] = None
    def lazy():
        """Loading in some of the problems is expensive so we should only do so if we are actually doing work"""
        nonlocal ts_problem
        ts_problem = tsp.TravellingSalesmanProblem[TREE_TYPE](TREE_TYPE, points, ax, s=S_FACTOR) if ts_problem is None else ts_problem

    # MARK: load the dp solution
    def try_load_dp_path(): # REVIEW: hacky
        num_path = None
        if os.path.exists(f"ALL_tsp/{problem.name}.opt.tour"): # If there is an optimal tour file
            try:
                opt_problem = tsplib95.load(f"ALL_tsp/{problem.name}.opt.tour")
            except:
                raise ValueError(f"Failed to load {problem.name} tour")
            assert(len(opt_problem.tours) == 1)
            assert(len(opt_problem.tours[0]) == problem.dimension)
            num_path = opt_problem.tours[0]
        if os.path.exists(f"ALL_tsp/{problem.name}.tour"): # If there is an optimal tour file
            try:
                non_opt_problem = tsplib95.load(f"ALL_tsp/{problem.name}.tour")
            except:
                raise ValueError(f"Failed to load {problem.name} tour")
            assert(len(non_opt_problem.tours) == 1)
            assert(len(non_opt_problem.tours[0]) == problem.dimension)
            num_path = non_opt_problem.tours[0]
        elif len(problem.tours) > 0:
            assert len(problem.tours) == 1
            num_path = problem.tours[0] # NOTE: I do not know of any euclidean which has this property
            raise ValueError("A shooting star")
        if num_path is not None:
            path = [points[i-1] for i in num_path] + [points[num_path[0] - 1]] # NOTE: ts_problem.points are reordered use the ordering from the files
            lazy()
            ts_problem.dp_path = (path, util.calc_dist(path), None)
            # ts_problem.draw_tour(ts_problem.dp_path[0], '#FFC0CB')
        elif len(points) <= BF_THRESHOLD:
            print("Brute forcing", problem.name)
            lazy()
            ts_problem.dp_alt_path = ts_problem.dp_path # Load the property
        else:
            return False
        return True
    
    new_problem = problem.name not in df.index
    if new_problem:
        print("New problem " + problem.name)

    # MARK: Populate row values
    if new_problem or pd.isna(df.loc[problem.name, "num_points"]):
        lazy()
        assert len(ts_problem.points) == len(points) and len(points) > 2, "Invalid points"
        updates["num_points"] = len(ts_problem.points)
    if new_problem or pd.isna(df.loc[problem.name, "opt_len"]):
        if try_load_dp_path():
            assert ts_problem.check_tour(ts_problem.dp_path[0]), "Invalid tour"
            updates["opt_len"] = ts_problem.dp_path[1]
    if new_problem or pd.isna(df.loc[problem.name, "nn_len"]):
        lazy()
        assert ts_problem.check_tour(ts_problem.nnn_path[0]), "Invalid tour"
        updates["nn_len"] = ts_problem.nnn_path[1]
    if new_problem or pd.isna(df.loc[problem.name, "untouched_len"]):
        lazy()
        assert ts_problem.check_tour(ts_problem.untouched_path[0]), "Invalid tour"
        updates["untouched_len"] = ts_problem.untouched_path[1]
    # if new_problem or pd.isna(df.loc[problem.name, "nwsp5_len"]) and "dp_path" in ts_problem.__dict__:
    #     lazy()
    #     assert ts_problem.check_tour(ts_problem.nwsp_path(5)[0]), "Invalid path"
    #     df.loc[problem.name, "nwsp5_len"] = ts_problem.nwsp_path(5)[1]
    # if new_problem or pd.isna(df.loc[problem.name, "ls_len"]):
    #     lazy()
    #     assert ts_problem.check_tour(ts_problem.local_search_path[0]), "Invalid tour"
    #     updates["ls_len"] = ts_problem.local_search_path[1]
    #     # save the tour to a file
    #     num_tour = ts_problem.point_tour_to_ids(ts_problem.local_search_path[0], offset_add=1)[:-1]
    #     tsplib95.models.StandardProblem(
    #         name=problem.name, 
    #         comment=f"Local search tour for {problem.name} ({ts_problem.quick_local_search_path[1]})",
    #         type="TOUR", 
    #         dimension=problem.dimension, 
    #         tours=[num_tour]
    #     ).save(f"SUPP_tsp/{problem.name}.ls.tour")

    # if new_problem or pd.isna(df.loc[problem.name, "ls_q_len"]):
    #     lazy()
    #     assert ts_problem.check_tour(ts_problem.quick_local_search_path[0]), "Invalid tour"
    #     updates["ls_q_len"] = ts_problem.quick_local_search_path[1]
    #     #save the tour to a file
    #     num_tour = ts_problem.point_tour_to_ids(ts_problem.quick_local_search_path[0], offset_add=1)[:-1]
    #     tsplib95.models.StandardProblem(
    #         name=problem.name, 
    #         comment=f"Quick (5min) local search tour for {problem.name} ({ts_problem.quick_local_search_path[1]})",
    #         type="TOUR", 
    #         dimension=problem.dimension, 
    #         tours=[num_tour]
    #     ).save(f"SUPP_tsp/{problem.name}.lsq.tour")
            
    if new_problem or pd.isna(df.loc[problem.name, f"wsp_count_{S_FACTOR}"]):
        lazy()
        assert len(ts_problem.wspd) > 0, "Invalid wspd"
        updates[f"wsp_count_{S_FACTOR}"] = len(ts_problem.wspd)
    
    # i += 1
    # if updates != {}:
    #     print(problem.name)
    
    return {problem.name: updates}

original_sigint_handler = signal.signal(signal.SIGINT, signal.SIG_IGN)
pool = Pool(6) # NUMBER OF PROCESSES
signal.signal(signal.SIGINT, original_sigint_handler)
update_collection = []
u = 0
pf = {"updated": 0}
try:
    with tqdm.tqdm(total=len(problems), unit="problem", postfix=pf) as pbar:
        for result in pool.imap_unordered(setup, problems):
            update_collection.append(result)
            pbar.update(1)
            prob_name = next(iter(result))
            if result[prob_name] != {}:
                u += 1
                pbar.set_postfix(pf := {**pf, **{"last": prob_name, "last_updated": prob_name, "updated": u}})
            else:
                pbar.set_postfix(pf := {**pf, **{"last": prob_name}})
except KeyboardInterrupt:
    print("Caught KeyboardInterrupt, terminating workers")
    pool.terminate()
else:
    pool.close()
pool.join()

print([x for x in update_collection if x is not None and x != {}])

for update in update_collection:
    if update is None: continue
    for key, value in update.items():
        df.loc[key, value.keys()] = value.values()
     
print("done") # TODO: use progress instead of prints

100%|██████████| 193/193 [00:05<00:00, 38.56problem/s, updated=3, last=zi929, last_updated=p654]      

[{'a280': {}}, {'bby34656': {}}, {'bch2762': {}}, {'bbz25234': {}}, {'bck2217': {}}, {'bcl380': {}}, {'berlin52': {}}, {'beg3293': {}}, {'bgb4355': {}}, {'bgd4396': {}}, {'bier127': {}}, {'bgf4475': {}}, {'bm33708': {}}, {'bnd7168': {}}, {'boa28924': {}}, {'bva2144': {}}, {'ch130': {}}, {'ch150': {}}, {'ca4663': {}}, {'d1291': {}}, {'d1655': {}}, {'d15112': {}}, {'d198': {}}, {'d2103': {}}, {'d493': {}}, {'d657': {}}, {'dbj2924': {}}, {'dca1389': {}}, {'dcb2086': {}}, {'dcc1911': {}}, {'dea2382': {}}, {'dga9698': {}}, {'dj38': {}}, {'dhb3386': {}}, {'dja1436': {}}, {'djb2036': {}}, {'djc1785': {}}, {'dka1376': {}}, {'dkc3938': {}}, {'dkd1973': {}}, {'brd14051': {'opt_len': 469842.24420658813}}, {'dke3097': {}}, {'dkf3954': {}}, {'dkg813': {}}, {'dlb3694': {}}, {'eg7146': {}}, {'eil101': {}}, {'eil51': {}}, {'eil76': {}}, {'ei8246': {}}, {'fdp3256': {}}, {'fea5557': {}}, {'d18512': {'opt_len': 645968.0153390475}}, {'fht47608': {}}, {'fi10639': {}}, {'fjr3672': {}}, {'fjs3649': {}}, {'fl

In [20]:
# df.iloc[-10:, :]
# df = df.replace({pd.NA: np.nan})
# df["wsp_count_2.0"] = df["wsp_count_2.0"].astype(pd.Int64Dtype())
print((~df.isna()).sum())
dfa = df[~df['ls_len'].isna()]
print((dfa['ls_len'] <= dfa['ls_q_len']).sum())
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(dfa)
df

num_points       208
opt_len          152
nn_len           206
untouched_len    208
nwsp5_2_len       65
nwsp5_len         17
wsp_count_2.0    208
ls_len            75
ls_q_len          75
dtype: int64
75


,num_points,opt_len,nn_len,untouched_len,nwsp5_2_len,nwsp5_len,wsp_count_2.0,ls_len,ls_q_len
name,,,,,,,,,
a280,280,2.586770e+03,3.161004e+03,4.847061e+03,NaN,NaN,2571,2769.255661,2956.917525
ara238025,238025,NaN,7.591498e+05,1.176390e+06,NaN,NaN,3032365,NaN,NaN
bby34656,34656,NaN,1.253833e+05,1.966541e+05,NaN,NaN,412676,NaN,NaN
bbz25234,25234,NaN,8.772034e+04,1.414553e+05,NaN,NaN,299703,NaN,NaN
bch2762,2762,NaN,1.073362e+04,1.624823e+04,NaN,NaN,28611,NaN,NaN
...,...,...,...,...,...,...,...,...,...
courbet180K,180000,7.894985e+06,9.407409e+06,1.868348e+09,NaN,NaN,3440440,NaN,NaN
earring200K,200000,8.178053e+06,9.762381e+06,2.188609e+09,NaN,NaN,3831148,NaN,NaN
lrb744710,744710,NaN,NaN,1.700243e+08,NaN,NaN,10051610,NaN,NaN


In [21]:
df.to_pickle('results.pkl')